### Agents

#### Agent Loop

At its core, the agent loop follows these steps:

* Receives user input and contextual information
* Processes the input using a language model (LLM)
* Decides whether to use tools to gather information or perform actions
* Executes tools and receives results
* Continues reasoning with the new information
* Produces a final response or iterates again through the loop

This cycle may repeat multiple times within a single user interaction, allowing the agent to perform complex, multi-step reasoning and autonomous behavior.

In [3]:
from strands import Agent
from strands.models.anthropic import AnthropicModel
from strands_tools import calculator

In [4]:
import os 

ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')

In [5]:
model = AnthropicModel(
    client_args = {
        'api_key': ANTHROPIC_API_KEY
    },
        max_tokens=1028,
    model_id="claude-sonnet-4-20250514",
    params={
        "temperature": 0.7,
    }
)

In [6]:
agent = Agent(model=model, tools=[calculator])

In [7]:
response = agent("what is 1 plus 5?")
response

I'll calculate 1 plus 5 for you.
Tool #1: calculator


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 1 + 5               │                                                                            │
│  │ Result    │ 6                   │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

1 plus 5 equals 6.

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': '1 plus 5 equals 6.'}]}, metrics=EventLoopMetrics(cycle_count=2, tool_metrics={'calculator': ToolMetrics(tool={'toolUseId': 'toolu_01QX34ci6Vu53j7fxK4HEzep', 'name': 'calculator', 'input': {'expression': '1 + 5'}}, call_count=1, success_count=1, error_count=0, total_time=0.03801274299621582)}, cycle_durations=[2.7241334915161133], traces=[<strands.telemetry.metrics.Trace object at 0x0000013C82267B10>, <strands.telemetry.metrics.Trace object at 0x0000013C81372C90>], accumulated_usage={'inputTokens': 3987, 'outputTokens': 82, 'totalTokens': 4069}, accumulated_metrics={'latencyMs': 0}), state={}, interrupts=None)

In [8]:
response = agent("what is 6 plus 5 and result multiply by 20?")
response

I'll calculate 6 plus 5 and then multiply that result by 20.
Tool #2: calculator


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ (6 + 5) * 20        │                                                                            │
│  │ Result    │ 220                 │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

6 plus 5 equals 11, and 11 multiplied by 20 equals 220.

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': '6 plus 5 equals 11, and 11 multiplied by 20 equals 220.'}]}, metrics=EventLoopMetrics(cycle_count=4, tool_metrics={'calculator': ToolMetrics(tool={'toolUseId': 'toolu_018keUV7neTmCGSuzfiSmYQM', 'name': 'calculator', 'input': {'expression': '(6 + 5) * 20'}}, call_count=2, success_count=2, error_count=0, total_time=0.04441976547241211)}, cycle_durations=[2.7241334915161133, 2.8766751289367676], traces=[<strands.telemetry.metrics.Trace object at 0x0000013C82267B10>, <strands.telemetry.metrics.Trace object at 0x0000013C81372C90>, <strands.telemetry.metrics.Trace object at 0x0000013C8232F350>, <strands.telemetry.metrics.Trace object at 0x0000013C8199F150>], accumulated_usage={'inputTokens': 8219, 'outputTokens': 189, 'totalTokens': 8408}, accumulated_metrics={'latencyMs': 0}), state={}, interrupts=None)

#### Structured Output

In [9]:
from pydantic import BaseModel, Field
from typing import Optional, List

class WeatherForecast(BaseModel):
    """Complete weather forecast information."""
    location: str = Field(description="The location for this forecast")
    current_time: str = Field(description="Current time in HH:MM format")
    current_weather: str = Field(description="Current weather conditions")
    temperature: Optional[float] = Field(default=None, description="Temperature in Celsius")
    forecast_days: List[str] = Field(default_factory=list, description="Multi-day forecast")

In [11]:
from pydantic import BaseModel
from strands import Agent

class PersonInfo(BaseModel):
    name: str
    age: int
    occupation: str

agent = Agent(model=model)
result = agent.structured_output(
    PersonInfo,
    "John Smith is a 30-year-old software engineer"
)

print(f"Name: {result.name}")      # "John Smith"
print(f"Age: {result.age}")        # 30
print(f"Job: {result.occupation}") # "software engineer"


Tool #1: PersonInfo
Name: John Smith
Age: 30
Job: software engineer


In [13]:
agent = Agent(model=model)

# Build up conversation context
agent("What do you know about Paris, France?")
agent("Tell me about the weather there in spring.")

# Extract structured information with a prompt
class CityInfo(BaseModel):
    city: str
    country: str
    population: Optional[int] = None
    climate: str

# Uses existing conversation context with a prompt
result = agent.structured_output(CityInfo, "Extract structured information about Paris")

Paris is the capital and largest city of France, with a rich history spanning over 2,000 years. Here are some key things about it:

**Geography & Population**
- Located in north-central France along the Seine River
- Population of about 2.1 million in the city proper, over 12 million in the metropolitan area
- Divided into 20 administrative districts called arrondissements

**Famous Landmarks**
- Eiffel Tower (built for the 1889 World's Fair)
- Notre-Dame Cathedral (currently being restored after the 2019 fire)
- Louvre Museum (world's largest art museum)
- Arc de Triomphe and Champs-Élysées
- Sacré-Cœur Basilica in Montmartre

**Culture & Arts**
- Major global center for art, fashion, and cuisine
- Home to numerous world-class museums and galleries
- Known for its café culture and culinary traditions
- Historic center is a UNESCO World Heritage Site

**Economy**
- Major financial and business hub
- Important for luxury goods, fashion, and tourism
- One of the world's most visited citi

In [14]:
result

CityInfo(city='Paris', country='France', population=2100000, climate='Temperate oceanic climate with mild springs, warm summers, cool autumns, and cold winters. Spring temperatures range from 41-68°F (5-20°C) with frequent rainfall.')

In [15]:
from typing import List
from pydantic import BaseModel, Field

class Address(BaseModel):
    street: str
    city: str
    country: str
    postal_code: Optional[str] = None

class Contact(BaseModel):
    email: Optional[str] = None
    phone: Optional[str] = None

class Person(BaseModel):
    """Complete person information."""
    name: str = Field(description="Full name of the person")
    age: int = Field(description="Age in years")
    address: Address = Field(description="Home address")
    contacts: List[Contact] = Field(default_factory=list, description="Contact methods")
    skills: List[str] = Field(default_factory=list, description="Professional skills")

agent = Agent(model=model)
result = agent.structured_output(
    Person,
    "Extract info: Jane Doe, a systems admin, 28, lives at 123 Main St, New York, NY. Email: jane@example.com"
)

print(result.name)                    # "Jane Doe"
print(result.address.city)            # "New York"
print(result.contacts[0].email)       # "jane@example.com"
print(result.skills)                  # ["systems admin"]


Tool #1: Person
Jane Doe
New York
jane@example.com
['systems admin']
